In [1]:
from selenium import webdriver  
import time  
import urllib  
from slugify import slugify
import requests
import random
import os

In [3]:
"""
siggraph urls examples

siggraph: https://dl.acm.org/toc/tog/2020/39/4 (SIGGRAPH 2020)
siggraph: https://dl.acm.org/toc/tog/2019/38/4 (SIGGRAPH 2019)
siggraph: https://dl.acm.org/toc/tog/2018/37/4 (SIGGRAPH 2018)
siggraph: https://dl.acm.org/toc/tog/2017/36/4 (SIGGRAPH 2017)

"""




'\nsiggraph urls examples\n\nsiggraph: https://dl.acm.org/toc/tog/2020/39/4 (SIGGRAPH 2020)\nsiggraph: https://dl.acm.org/toc/tog/2019/38/4 (SIGGRAPH 2019)\nsiggraph: https://dl.acm.org/toc/tog/2018/37/4 (SIGGRAPH 2018)\nsiggraph: https://dl.acm.org/toc/tog/2017/36/4 (SIGGRAPH 2017)\n\n'

In [2]:

"""
some variables needed to be set up by users
"""
conference = 'siggraph'
conference_url = "https://dl.acm.org/toc/tog/2021/40/4" # the conference url to download papers from
edgedriver_path = 'C:\\Users\\xiaoyang\\Downloads\\edgedriver_win64\msedgedriver.exe' # the chromedriver.exe path
root = r'E:\OneDrive\academics\papers\conferences\SIGGRAPH-2021-ALL'.replace('\\','/') # file path to save the downloaded papers

os.makedirs(root, exist_ok=True)
print(root)

E:/OneDrive/academics/papers/conferences/SIGGRAPH-2021-ALL


In [13]:
session_1 = driver.find_elements_by_class_name('accordion-tabbed')[1].find_elements_by_class_name('toc__section')[0]

In [30]:
(session_1.find_element_by_xpath('a').text.split(':')[1].lstrip())

'Imaginative imaging'

In [21]:
session_1.find_elements_by_class_name('issue-item__content')[0].find_element_by_xpath('div/h5').text

'Session details: Imaginative imaging'

In [4]:
def retrieve_from_siggraph(driver):
    pdfurllist =  []
    pdfnamelist = []
    sessions = []
    import time    
    elementllist =  driver.find_elements_by_class_name('accordion-tabbed')[1].find_elements_by_class_name('toc__section')
    num_papers = 0
    for i, session in enumerate(elementllist):
        session.click()
        time.sleep(3)
        
        session_list = {}
        session_list['session'] = session.find_element_by_xpath('a').text.split(':')[1].lstrip()
        session_list['pdfnamelist'] = []
        session_list['pdfurllist'] = []
        print('\nSession: ', session_list['session'])

        for j, paper_element in enumerate(session.find_elements_by_class_name('issue-item__content')):
            try:
                paper_name = paper_element.find_element_by_xpath('div/h5').text
                pdf_url = paper_element.find_element_by_class_name('red').get_attribute('href')
                print('\t', paper_name)
                session_list['pdfnamelist'].append(paper_name)
                session_list['pdfurllist'].append(pdf_url)
                num_papers += 1
            except Exception as e:
                print(e)
            
        sessions.append(session_list)
    return sessions, num_papers

In [5]:
driver = webdriver.Edge(edgedriver_path)  
driver.get(conference_url)

print('Retrieving pdf urls. This could take some time...')
sessions, num_papers = retrieve_from_siggraph(driver)



Retrieving pdf urls. This could take some time...

Session:  Denoising and guiding
	 Hierarchical neural reconstruction for path guiding using hybrid path and photon samples
	 Real-time neural radiance caching for path tracing
	 Interactive Monte Carlo denoising using affinity of neural features
	 Weakly-supervised contrastive learning in path manifold for Monte Carlo image reconstruction

Session:  Fabrication 1
	 Bistable auxetic surface structures
	 Computational inverse design of surface-based inflatables
	 Modeling and fabrication with specified discrete equivalence classes
	 Using isometries for computational design and fabrication

Session:  Portrait editing
	 Total relighting: learning to relight portraits for background replacement
	 PhotoApp: photorealistic appearance editing of head portraits
	 Only a matter of style: age transformation using a style-based regression model
	 Coarse-to-fine: facial structure editing of portrait images via latent space classifications

Session

In [6]:
# check the retrieved urls
print('The first pdf urls:\n')
print(sessions[0]['pdfurllist'])
print('\nThe last pdf urls:\n')
print(sessions[-1]['pdfurllist'])


The first pdf urls:

['https://dl.acm.org/doi/pdf/10.1145/3450626.3459810', 'https://dl.acm.org/doi/pdf/10.1145/3450626.3459812', 'https://dl.acm.org/doi/pdf/10.1145/3450626.3459793', 'https://dl.acm.org/doi/pdf/10.1145/3450626.3459876']

The last pdf urls:

['https://dl.acm.org/doi/pdf/10.1145/3450626.3459671', 'https://dl.acm.org/doi/pdf/10.1145/3450626.3459802']


In [7]:
# check the retrieved paper titles

print('The first pdf titles:\n')
print(sessions[0]['pdfnamelist'])
print('\nThe last pdf titles:\n')
print(sessions[-1]['pdfnamelist'])


The first pdf titles:

['Hierarchical neural reconstruction for path guiding using hybrid path and photon samples', 'Real-time neural radiance caching for path tracing', 'Interactive Monte Carlo denoising using affinity of neural features', 'Weakly-supervised contrastive learning in path manifold for Monte Carlo image reconstruction']

The last pdf titles:

['QuanTaichi: a compiler for quantized simulations', 'Intersection-free rigid body dynamics']


In [8]:
print('The number of papers is ', num_papers)


The number of papers is  143


In [14]:
# download the papers one by one. The files are named after the titles (guaranteed to be valid file name after processed by slugify.)
print('Start downloading')
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:57.0) Gecko/20100101 Firefox/57.0'}
i = 0
for session in sessions:
    pdfurllist = session['pdfurllist']
    pdfnamelist = session['pdfnamelist']
    session_dir = slugify(session['session'])
    os.makedirs(root+'/'+session_dir, exist_ok=True)
    for j, url in enumerate(pdfurllist):
        if url != None :      
            pdfname = slugify(pdfnamelist[j])
            if os.path.isfile(root+'/'+session_dir+'/'+pdfname+".pdf"):
                print('existed', i, '\t', pdfnamelist[j], '\t', pdfurllist[j])
            else:
                print(i, '\t', pdfnamelist[j], '\t', pdfurllist[j])
                
                try_download = True
                while try_download:
                    try:
                        data = requests.get(pdfurllist[j], timeout=80, headers=headers).content
                        try_download = False
                    except ConnectionError as e: 
                        _ = time.sleep(random.uniform(1,2)) # for anti-anti-crawler purpose
                        
                with open(root+'/'+session_dir+'/'+pdfname+".pdf", 'wb')  as f:
                    f.write(data)  
                    
            i += 1

Start downloading
existed 0 	 Hierarchical neural reconstruction for path guiding using hybrid path and photon samples 	 https://dl.acm.org/doi/pdf/10.1145/3450626.3459810
existed 1 	 Real-time neural radiance caching for path tracing 	 https://dl.acm.org/doi/pdf/10.1145/3450626.3459812
existed 2 	 Interactive Monte Carlo denoising using affinity of neural features 	 https://dl.acm.org/doi/pdf/10.1145/3450626.3459793
existed 3 	 Weakly-supervised contrastive learning in path manifold for Monte Carlo image reconstruction 	 https://dl.acm.org/doi/pdf/10.1145/3450626.3459876
4 	 Bistable auxetic surface structures 	 https://dl.acm.org/doi/pdf/10.1145/3450626.3459940
5 	 Computational inverse design of surface-based inflatables 	 https://dl.acm.org/doi/pdf/10.1145/3450626.3459789
6 	 Modeling and fabrication with specified discrete equivalence classes 	 https://dl.acm.org/doi/pdf/10.1145/3450626.3459843
7 	 Using isometries for computational design and fabrication 	 https://dl.acm.org/doi/